In [1]:
import pandas as pd
import numpy as np
import shap
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, BaseCrossValidator
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error, accuracy_score, median_absolute_error, make_scorer
import xgboost
from scipy.stats import loguniform, randint, uniform

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.early_stop import no_progress_loss
from hyperopt.pyll.base import Apply

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt.plots import plot_convergence

from missforest.missforest import MissForest

#Creating a custom time series cross-validator
class CustomTimeSeriesCV(BaseCrossValidator):
    """Creates an iterator that contains the indices from each dataset based on the years given"""
    def __init__(self, years):
        self.years = years

    def split(self, X, y=None, groups=None):
        for train_years, test_years in self.years:
            train_indices = np.where(X['year'].isin(train_years))[0]
            test_indices = np.where(X['year'].isin(test_years))[0]
            yield train_indices, test_indices
        
    def get_n_splits(self, X=None, y=None, groups=None):
        return len(self.years)

class MissForestImputer(BaseEstimator, TransformerMixin):
    """Probably useless class -- theoretically uses MissForest with sklearn's API, but it's not working as expected.
    For some reason, LightGBM isn't able to create trees on the missing data"""
    def __init__(self, max_iter = 20):
        self.model = MissForest(max_iter = max_iter)
    
    def get_params(self, deep: bool = False) -> dict:
        return {'max_iter': self.model.max_iter}
    
    def fit(self, X, y=None):
        self.model.fit(X)
        return self  # Return self to enable chaining

    def transform(self, X):
        # MissForest's transform method is actually predict in most cases
        X_imputed = self.model.transform(X)
        return X_imputed

    def fit_transform(self, X, y=None, **fit_params):
        return self.model.fit_transform(X)
 
    
def missing_kernel(X, Y):
    """Kernel method that works with missing values in the data. It computes the RBF kernel between two matrices X and Y.
    Theoretically, it should work for sklearn's SVM, but I can't get it to work with gamma/C parameters."""
    # Calculate pairwise squared differences using broadcasting, with an extra dimension for vectorization
    diffs = X[:, np.newaxis, :] - Y[np.newaxis, :, :]
    
    # Create masks for non-missing values (True for non-missing)
    non_missing_mask_X = ~np.isnan(X)[:, np.newaxis, :]
    non_missing_mask_Y = ~np.isnan(Y)[np.newaxis, :, :]
    
    # Only consider differences where both corresponding values are non-missing
    valid_diffs_mask = non_missing_mask_X & non_missing_mask_Y
    valid_diffs = np.where(valid_diffs_mask, diffs, 0)
    
    # Compute squared differences
    squared_diffs = valid_diffs ** 2
    
    # Count valid (non-missing) comparisons for normalization
    valid_counts = np.sum(valid_diffs_mask, axis=2)
    
    # Avoid division by zero for rows/columns with all missing values
    valid_counts[valid_counts == 0] = 1
    
    # Compute gamma as the inverse of valid comparisons
    gamma = 1 / valid_counts
    
    # Sum squared differences along the feature dimension
    sum_squared_diffs = np.sum(squared_diffs, axis=2)
    
    # Compute RBF kernel values
    M = np.exp(-gamma * sum_squared_diffs)
    
    return M

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [12]:
def optima_model(model, param_dict, X, y, classify = True):
    """Performs hyperparameter optimization for a a given model, keeping track of loss. 
    ## Parameters:
    model: sklearnable model, like XGBoost or Linreg
    param_dict: dictionary of hyperparameters to optimize
    X: DataFrame with features
    y: Series with target variable
    classify: Boolean, whether we are using a classifier or regressor"""
    
    if classify:
        #When classifying, y must be boolean
        y = y.where(y > 0, 0)
        y = y.where(y <= 0, 1)
        
    X_train, X_test, y_train, y_test = (X.loc[X['year'] < 2022, :], X.loc[X['year'] == 2022, :], 
                                        y.loc[X['year'] < 2022], y.loc[X['year'] == 2022])
    

    # Create fold structure so we can make a custom cross-validation for time-series
    folds = [
        (range(2002, 2006, 2), [2006, 2008]),
        (range(2002, 2010, 2), [2010, 2012]),
        (range(2002, 2014, 2), [2014, 2016]),
        (range(2002, 2018, 2), [2018, 2020])
    ]

    cv = CustomTimeSeriesCV(folds)
        
    #Categorical features that need to be one-hot encoded    
    one_hot_fts = ['office_type', 'open_seat', 'special', 'isMidterm']
    
    #Cont features that should be pass-throughed (and later scaled)
    cont_fts = ['incumbent_differential', 'prev_gen_margin', 'prev2_gen_margin',
    'prev_dem_gen_tp', 'mean_specials_differential', 'pvi', 'weighted_genpoll', 
    'unweighted_genpoll', 'genballot_predicted_margin', 'specials_predicted_margin',
    'house_chamber_margin', 'senate_chamber_margin', 'previous_cci',
    'current_cci', 'change_cci', 'previous_gas', 'current_gas',
    'change_gas', 'previous_unemployment', 'current_unemployment',
    'change_unemployment', 'white_pct', 'black_pct', 'asian_pct', 'impoverished_pct', 
    'median_age', 'renting_pct', 'inflation', 'receipts_DEM', 'receipts_REP',
    'disbursements_DEM', 'disbursements_REP', 'receipts_ratio', 'disbursements_ratio', 
    'unconvinced_pct', 'valid_weighted_ba', 'phone_unweighted_ba', 'online_unweighted_ba',
    'all_unweighted_ba', 'all_unweighted', 'num_polls', 
    "absenteeexcusereq", "pollhours", "avgpollhours",
    "minpollhours", "regdeadlines", "voteridlaws", "novoterid",
    "noallmailvote", "noearlyvote", "nofelonreg", "nofelonsregafterincar",
    "nonstrictid", "nonstrictphoto", "nopollplacereg", "nopr",
    "nosamedayreg", "nostateholiday", "pr16", "pr17",
    "pr175", "pr60", "pr90", "strictid",
    "strictphoto", "covi_num", "median_income"]
        
    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(), one_hot_fts),
        ('num', 'passthrough', cont_fts)])
    
    
    def objective(params):
        "Function that takes in hyperparameters and returns loss, that Hyperopt will minimize."
        reg = model(**params)
        pipe = Pipeline(steps = [
            ('preprocessing', preprocessor), 
            ('scaling', StandardScaler()),
            #('polynomial', PolynomialFeatures(degree=2)), Use only if you want to add polynomial features
            ('model', reg)])
        
        
        training_loss = []
        testing_loss = []
        for train_idx, test_idx in cv.split(X_train):
            """Goes through each fold and calculates loss.
            Note: We use median absolute error because it is more robust to outliers than mean absolute error.
            We also expect earlier folds to have higher error, since they have less data to train on."""
            pipe.fit(X_train.iloc[train_idx], y_train.iloc[train_idx])
            
            predictions = pipe.predict(X_train.iloc[test_idx])
            training_preds = pipe.predict(X_train.iloc[train_idx])
            if classify:
                "We multiply by -1 because hyperopt minimizes the loss, but we want to maximize accuracy"
                testing_loss.append(-1 * accuracy_score(y_train.iloc[test_idx], predictions))
                training_loss.append(-1 * accuracy_score(y_train.iloc[train_idx], training_preds))
            else:
                testing_loss.append(median_absolute_error(y_train.iloc[test_idx], predictions))
                training_loss.append(median_absolute_error(y_train.iloc[train_idx], training_preds))
            
        print(f"Training loss: {training_loss}")
        print(f"Validation loss: {testing_loss}")
        print(f"Overfitting differential: {np.mean(testing_loss) - np.mean(training_loss)}")
        return {'loss': np.mean(testing_loss), 'status': STATUS_OK}


    "Hyperopt uses the TPE algorithm to optimize hyperparameters. We use the no_progress_loss function to stop early if we don't see progress."
    best_params = fmin(fn=objective,
                    space=param_dict,
                    algo=tpe.suggest,
                    trials=Trials(),
                    early_stop_fn=no_progress_loss(iteration_stop_count=10))

    print("Best parameters:", best_params)
    
    acc_model = model(**best_params)
    
    "This works with SHAP to calculate importance values. Shap can't handle pipelines, so we need to preprocess the data first."
    preprocessed_X_train = preprocessor.fit_transform(X_train)
    scaled_X_train = StandardScaler().fit_transform(preprocessed_X_train)
    
    acc_model.fit(scaled_X_train, y_train)
    "Change this to TreeExplainer if you are using a tree-based model"
    #Explainer = shap.Explainer(acc_model)
    Explainer = shap.TreeExplainer(acc_model)
    shap_values = Explainer.shap_values(scaled_X_train)
    shap.summary_plot(shap_values, pd.DataFrame(data = scaled_X_train, columns = preprocessor.get_feature_names_out()), plot_type="bar") 
    "Returns a dataframe with SHAP values for future analysis"   
    return pd.DataFrame(shap_values, columns = preprocessor.get_feature_names_out())


In [5]:
import re
#methods to impiute, but can't get any to work right now...
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler


data = pd.read_csv("../cleaned_data/Engineered Dataset.csv")
data = data.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

X = data.drop(columns=['margin'])
y = data['margin']


#This imputation method doesn't work :(
#We want to impute BEFORE we actually run the data through the model, being sure to not leak data beforehand
"""
one_hot_fts = ['office_type', 'open_seat', 'special', 'isMidterm', 'state', 'year', 'district', 'final_rating']

imputing_columns = [x for x in X.columns if x not in one_hot_fts]
bi = BiScaler()
bi.fit(np.array(X[X['year'] < 2006][imputing_columns]))
data_pre_2006 = bi.transform(np.array(X[X['year'] < 2006][imputing_columns]))
data_2006 = bi.transform(np.array(X[X['year'] == 2006][imputing_columns]))
bi.fit(np.array(X[X['year'] < 2010][imputing_columns]))
data_2008 = bi.transform(np.array(X[X['year'] == 2008][imputing_columns]))
data_2010 = bi.transform(np.array(X[X['year'] == 2010][imputing_columns]))
bi.fit(np.array(X[X['year'] < 2014][imputing_columns]))
data_2012 = bi.transform(np.array(X[X['year'] == 2012][imputing_columns]))
data_2014 = bi.transform(np.array(X[X['year'] == 2014][imputing_columns]))
bi.fit(np.array(X[X['year'] < 2018][imputing_columns]))
data_2016 = bi.transform(np.array(X[X['year'] == 2016][imputing_columns]))
data_2018 = bi.transform(np.array(X[X['year'] == 2018][imputing_columns]))
bi.fit(np.array(X[X['year'] < 2022][imputing_columns]))
data_2020 = bi.transform(np.array(X[X['year'] == 2020][imputing_columns]))
data_2022 = bi.transform(np.array(X[X['year'] == 2022][imputing_columns]))
bi.fit(np.array(X[X['year'] < 2024][imputing_columns]))
data_2024 = bi.transform(np.array(X[X['year'] == 2024][imputing_columns]))
"""

"\none_hot_fts = ['office_type', 'open_seat', 'special', 'isMidterm', 'state', 'year', 'district', 'final_rating']\n\nimputing_columns = [x for x in X.columns if x not in one_hot_fts]\nbi = BiScaler()\nbi.fit(np.array(X[X['year'] < 2006][imputing_columns]))\ndata_pre_2006 = bi.transform(np.array(X[X['year'] < 2006][imputing_columns]))\ndata_2006 = bi.transform(np.array(X[X['year'] == 2006][imputing_columns]))\nbi.fit(np.array(X[X['year'] < 2010][imputing_columns]))\ndata_2008 = bi.transform(np.array(X[X['year'] == 2008][imputing_columns]))\ndata_2010 = bi.transform(np.array(X[X['year'] == 2010][imputing_columns]))\nbi.fit(np.array(X[X['year'] < 2014][imputing_columns]))\ndata_2012 = bi.transform(np.array(X[X['year'] == 2012][imputing_columns]))\ndata_2014 = bi.transform(np.array(X[X['year'] == 2014][imputing_columns]))\nbi.fit(np.array(X[X['year'] < 2018][imputing_columns]))\ndata_2016 = bi.transform(np.array(X[X['year'] == 2016][imputing_columns]))\ndata_2018 = bi.transform(np.array(X

In [ ]:
#Running xgb with hyperopt
param_dist_xgb = {
    'n_estimators': hp.randint('n_estimators', 5, 300),
    'max_depth': hp.randint('max_depth', 10, 100),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.2)),
    'subsample': hp.uniform('subsample', 0., 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1),
    'min_child_weight': hp.randint('min_child_weight', 1, 20),
    'gamma': hp.loguniform('gamma', np.log(5), np.log(20)),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(0.01), np.log(100)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(0.01), np.log(100))
}


xgb = xgboost.XGBClassifier
optima_model(xgb, param_dist_xgb, X, y, True)